In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=f16897685661c0684e3080a50c920cf34b2d526130a593ed3787c2ff9ccc0859
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0285 ,random_state=0 , l1_ratio=0.6432, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 6.40 , C = 365.65 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_13_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=893877, Mon Apr 13 14:47:20 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 7.741269651966092e+03 1.0e+00 2.55e-01  3e-01  3e-01 0:00.0
    2     30 7.741263575547718e+03 1.0e+00 2.48e-01  2e-01  2e-01 0:00.0
    3     45 7.741258766969286e+03 1.1e+00 2.43e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7741.30729349] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7741.31855163] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 3.556576688948165e+03 1.5e+00 4.94e-02  5e-02  5e-02 0:01.9
  200   3000 3.161919740552764e+03 1.5e+00 3.23e-03  3e-03  3e-03 0:03.8
  300   4500 3.160081945726964e+03 1.6e+00 2.18e-04  2e-04  2e-04 0:05.6
  400   6000 3.160076317731175e+03 1.6e+00 1.47e-05  1e-05  1e-05 0:07.5
  500   7500 3.160076282256422e+03 1.6e+00 9.14e-07  7e-07  8e-07 0:09.4
  600   9000 3.160076282154881e+03 1.7e+00 5.37e-08  4e-08  5e-08 0:11.3
  700  10500 3.160076282154425e+03 1.7e+00 4.50e-09  3e-09  4e-09 0:13.1
  780  11700 3.160076282154423e+03 1.8e+00 1.99e-09  1e-09  2e-09 0:14.6
termination on tolfun=1e-11 (Mon Apr 13 14:47:37 2020)
final/bestever f-value = 3.160076e+03 3.160076e+03
incumbent solution: [-0.03729192  0.16792745 -0.24733188  0.21473333  0.1418224  -0.02816852
 -0.12247467  0.07988005 ...]
std deviations: [1.51945137e-09 1.40648199e-09 1.58403568e-09 1.58463494e-09
 1.47156062e-09 1.63854113e-09 1.59610598e-09 1.49870859e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7748.04869204] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7748.0486904] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 7.505287786437639e+03 1.4e+00 2.55e-02  2e-02  3e-02 0:01.3
  200   3000 7.485731714997612e+03 1.5e+00 1.25e-03  1e-03  1e-03 0:02.7
  300   4500 7.485681456772003e+03 1.5e+00 7.50e-05  6e-05  7e-05 0:04.0
  400   6000 7.485681309009656e+03 1.5e+00 5.04e-06  4e-06  5e-06 0:05.3
  500   7500 7.485681308470898e+03 1.6e+00 3.55e-07  3e-07  3e-07 0:06.6
  600   9000 7.485681308468722e+03 1.6e+00 2.16e-08  2e-08  2e-08 0:08.0


...

  100   1500 7.517811326039877e+03 1.4e+00 2.44e-02  2e-02  2e-02 0:01.3
  200   3000 7.506716782211991e+03 1.4e+00 1.36e-03  1e-03  1e-03 0:02.7
  300   4500 7.506683773302469e+03 1.4e+00 7.89e-05  7e-05  7e-05 0:04.0
  400   6000 7.506683665093630e+03 1.5e+00 4.50e-06  4e-06  4e-06 0:05.3
  500   7500 7.506683664722289e+03 1.6e+00 3.32e-07  3e-07  3e-07 0:06.7
  600   9000 7.506683664720566e+03 1.6e+00 2.59e-08  2e-08  2e-08 0:08.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=7506.683665, sigma=8.09e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=643)
  ')')


  700  10500 7.506683664720559e+03 1.7e+00 4.87e-09  4e-09  4e-09 0:09.3
  709  10635 7.506683664720559e+03 1.7e+00 5.06e-09  4e-09  4e-09 0:09.5
termination on tolfun=1e-11 (Mon Apr 13 15:02:29 2020)
final/bestever f-value = 7.506684e+03 7.506684e+03
incumbent solution: [-0.03043294  0.07527816 -0.17371063  0.12669349  0.12777737 -0.06348448
 -0.05164055  0.00230375 ...]
std deviations: [4.10330102e-09 3.91613549e-09 3.99802960e-09 4.07147414e-09
 3.96307876e-09 4.24574606e-09 3.93707619e-09 3.98962073e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=859809, Mon Apr 13 15:02:29 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 8.153014007917244e+03 1.0e+00 2.54e-01  3e-01  3e-01 0:00.0
    2     30 7.687504478381024e+03 1.0e+00 2.44e-01  2e-01  2e-01 0:00.0
    3     45 7.522965590553720e+03 1.1e+00 2.38e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8826.89834418] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7687.50447838] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.229301020909008e+09 1.9e+00 1.19e+02  1e+02  1e+02 0:01.8
  110   1650 -5.476124298098316e+09 1.9e+00 2.74e+02  3e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 15:02:31 2020)
final/bestever f-value = -5.489658e+09 -5.489658e+09
incumbent solution: [-576.54956265 1504.22571206 -789.02724897  488.43050579  891.07459714
 -852.33459153 -784.1565518  -407.78609369 ...]
std deviations: [267.62803995 261.93050802 273.09598396 270.92900429 272.70574189
 273.32349153 276.86393651 261.4506844  ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F13_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
KNN_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:50])
    SVM_Fun [i] = F13(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F13(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([3157.39032352]), count=array([1])) 9.287160478897406e-07
SVM
ModeResult(mode=array([3009.4942693]), count=array([1])) 72.04115004257552
ELN
ModeResult(mode=array([933576.2981575]), count=array([1])) 2445228.9849652164
